In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

# Embeddings from Resnet18

In [2]:
# Load a pre-trained ResNet model (resnet18)
model = models.resnet18(pretrained=True)

/home/saul/.pyenv/versions/3.11.8/envs/llm/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/saul/.pyenv/versions/3.11.8/envs/llm/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Remove the last fully connected layer to get the embeddings
model = nn.Sequential(*list(model.children())[:-1])

In [4]:
# Set model to evaluation mode
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [5]:
# Define the image preprocessing transformations
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [7]:
# Load an image from a URL or local path (replace URL with your image path if needed)
url = "https://images.unsplash.com/photo-1593642632823-8f785ba67e45?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=MnwzNjUyOXwwfDF8c2VhcmNofDF8fHRlY2h8ZW58MHx8fHwxNjc5MjA2NzQ2&ixlib=rb-1.2.1&q=80&w=400"  # Replace with a valid image URL or local file path
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [8]:
# Preprocess the image and add a batch dimension
input_tensor = preprocess(img).unsqueeze(0)  # Shape: [1, 3, 224, 224]

In [10]:
# Generate embeddings by passing the input through the model
with torch.no_grad():
    embedding = model(input_tensor)
print(f"Embedding shape: {embedding.shape}")

Embedding shape: torch.Size([1, 512, 1, 1])


In [11]:
# Remove the extra dimensions to view as a vector
embedding_vector = embedding.flatten()

print(f"Embedding vector shape: {embedding_vector.shape}")
print(f"Embedding vector:\n{embedding_vector}")

Embedding vector shape: torch.Size([512])
Embedding vector:
tensor([2.8576e+00, 1.1922e+00, 3.4011e-01, 6.4422e-01, 2.0134e-01, 2.6692e-01,
        4.9765e-01, 1.1040e-01, 1.8118e+00, 8.5407e-01, 1.2696e+00, 1.8353e+00,
        7.5778e-01, 1.1389e+00, 3.9937e-02, 1.8792e+00, 5.1544e-02, 1.5950e-01,
        6.5171e-01, 1.5792e+00, 5.2563e-02, 2.1139e-01, 1.1726e+00, 1.7703e+00,
        1.5273e+00, 1.3297e+00, 5.5108e-01, 2.4302e+00, 1.3671e-01, 2.4126e-01,
        1.1358e+00, 7.5909e-01, 4.2556e-01, 4.9568e-01, 1.1930e-01, 6.9830e-01,
        7.8944e-01, 3.6795e-01, 2.0914e+00, 6.5689e-01, 1.5763e+00, 2.3488e-01,
        4.6677e-01, 1.8104e-01, 1.1846e+00, 1.1247e+00, 5.9191e-01, 1.5561e+00,
        1.1378e+00, 1.0383e+00, 5.4494e-01, 1.5300e+00, 2.1667e-01, 1.2570e-01,
        6.4365e-01, 6.6463e-01, 0.0000e+00, 6.5498e-01, 5.5730e-01, 2.6462e-01,
        3.9504e-01, 1.0762e-02, 1.2542e-02, 7.9144e-01, 1.1295e+00, 7.7308e-02,
        3.6684e-01, 3.8991e-01, 5.6734e-02, 4.3304e-01, 6.32